In [31]:
import requests #http://docs.python-requests.org/en/master/ 
from bs4 import BeautifulSoup
from datetime import datetime as dt
import time
import pandas as pd

In [43]:
url = "https://www.uspto.gov/about-us/events"
headers = {"User-Agent":"kekoziar_ptrc/0.0 (Language=Python/3.9.7; Host=https://github.com/kekoziar/ptrc; email=katherine.koziar@ucr.edu) python-requests/2.26.0"}
events = pd.DataFrame(columns = ['title', 'link', 'pubDate', 'description', 'page_owner','date_index'])

In [33]:
webpage = requests.get(url, headers=headers)

In [34]:
soup = BeautifulSoup(webpage.content, 'html.parser')
tr = soup.find_all('tr')

# Create dataframe

In [44]:
# this checks if the workshop is after the current date
url_start = "https://www.uspto.gov"
current_date = 20220125
date = 20221231

index = len(tr)-1

while(date >= current_date):
#    print(index)
    if(str(tr[index]).find("date=")!=-1):
        string = str(tr[index])
        date_index=(string.find("date="))+6
        date = int(string[date_index:date_index+8])
        workshop_url = tr[index].find(href=True)['href']
#        print("link: ", workshop_url)

        webpage = requests.get(url_start+workshop_url, headers = headers)
        soup_workshop = BeautifulSoup(webpage.content, 'html.parser')
        
        workshop_title = soup_workshop.find('meta', {"name":"dcterms.title"})
        workshop_pageowner = soup_workshop.find('meta', {"name":"uspto.pageowner"})
        
        workshop_description = soup_workshop.select_one('var.atc_description').text
        workshop_description = workshop_description[0:workshop_description.find('\n')].strip().replace('\xa0', ' ')
        workshop_description = workshop_description.replace('\u200b','')

        workshop_time = soup_workshop.select_one('var.atc_date_start').text
        workshop_time = workshop_time.replace(" ", "T")
        workshop_time = dt.fromisoformat(workshop_time)
        
        workshop_info = pd.DataFrame({
            "title": [workshop_title["content"]], 
            "link": [url_start+workshop_url], 
            "pubDate": [workshop_time.strftime("%a, %d %b %Y %X EST")], 
            "description": [workshop_description],
            "page_owner": [workshop_pageowner["content"]], 
            "date_index": [date]
        })        
        new = [events, workshop_info]
        events = pd.concat(new)
        time.sleep(1)
         
#    else:
#        print("not here")
    index = index - 1

In [48]:
events = events.sort_values(by=["date_index"])
events.to_csv("../data/events.csv")

In [9]:
# this checks if the workshop is after the current date
url_start = "https://www.uspto.gov"
current_date = 20220122
# date = 20221231
date = 20220201
index = len(tr)-1
while(date >= current_date):
    print(index)
    if(str(tr[index]).find("date=")!=-1):
        string = str(tr[index])
        date_index=(string.find("date="))+6
        print(date_index)
        print("iso8601 date: ",string[date_index:date_index+8])
        date = int(string[date_index:date_index+8])
        url_workshop = tr[index].find(href=True)['href']
        print("link: ", url_workshop)
        # print("workshop name: ", tr[index].select_one('a').text)
        # store this info in a dataframe
        # pull time info from a dataframe. get from description file, <div class="event-info">
        # or <var class="atc_date_start"> <-- probbly get from that, since date is already in 
        # eastern time
         
    else:
        print("not here")
    index = index - 1
    
    
    

3037
213
iso8601 date:  20221201
link:  /about-us/events/intellectual-property-basics-and-helpful-resources-10
workshop name:  Intellectual property basics and helpful resources
3036
not here
3035
213
iso8601 date:  20221103
link:  /about-us/events/intellectual-property-basics-and-helpful-resources-9
workshop name:  Intellectual property basics and helpful resources
3034
not here
3033
149
iso8601 date:  20221026
link:  /learning-and-resources/kids-educators/k-12-ip-education-office-hours-17
workshop name:  K-12 IP education office hours 
3032
149
iso8601 date:  20221025
link:  /learning-and-resources/kids-educators/intellectual-property-ip-workshop-k12-educators-10
workshop name:  Intellectual Property (IP) Workshop for K12 Educators 
3031
213
iso8601 date:  20221006
link:  /about-us/events/intellectual-property-basics-and-helpful-resources-8
workshop name:  Intellectual property basics and helpful resources
3030
not here
3029
149
iso8601 date:  20220928
link:  /learning-and-resources/

In [17]:
        # description pull this later, getting url_workshop from dataframe
        webpage = requests.get(url_start+url_workshop, headers=headers)
        soup_workshop = BeautifulSoup(webpage.content, 'html.parser')
        workshop_description = soup_workshop.select_one('var.atc_description').text
        xml_description = workshop_description[0:workshop_description.find('\n')].strip().replace('\xa0', ' ')
        print("description: ", xml_description)
        #time.sleep(5)


description:  Unfamiliar with patents, trademarks, copyrights, and trade secrets? Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and potential ways to protect your innovation as you transition from idea to product.


In [47]:
soup_workshop.select_one('var.atc_date_start').text

'2022-01-20 20:00:00'

In [19]:
import datetime

In [48]:
workshop_time = soup_workshop.select_one('var.atc_date_start').text

In [49]:
workshop_time = workshop_time.replace(" ", "T")

In [50]:
workshop_time = datetime.datetime.fromisoformat(workshop_time)

In [51]:
# workshop_time.strftime("%c")

'Thu Jan 20 20:00:00 2022'

In [52]:
#  Sat, 22 Jan 2022 10:30:00 PST
print(workshop_time.strftime("%a, %d %b %Y %X EST"))

Thu, 20 Jan 2022 20:00:00 EST


In [19]:
url_start = "https://www.uspto.gov"
workshop_url = "/about-us/events/intellectual-property-basics-and-helpful-resources-10"

In [22]:
webpage = requests.get(url_start+workshop_url, headers=headers)


In [23]:
soup_workshop = BeautifulSoup(webpage.content, 'html.parser')
# workshop_description = soup_workshop.select_one('var.atc_description').text

In [83]:
workshop_description.find('\n')

295

In [89]:
workshop_description[0:workshop_description.find('\n')].strip().replace('\xa0', ' ')

'Unfamiliar with patents, trademarks, copyrights, and trade secrets? Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and potential ways to protect your innovation as you transition from idea to product.'

In [77]:
soup_workshop.find_all(class_="atc_description")

[<var class="atc_description"> Unfamiliar with patents, trademarks, copyrights, and trade secrets? Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and potential ways to protect your innovation as you transition from idea to product. 
 The USPTO offers this free event for aspiring entrepreneurs, innovators, and students the first Thursday of every month. 
 Register today 
 The discussion will cover:
 An overview of intellectual property types: patents, trademarks, copyrights, and trade secrets	Why innovators and entrepreneurs should consider protecting their IP	Local resources and assistance available through the USPTO and other agencies
 For questions or more information, email Events@uspto.gov. </var>]

In [80]:
soup_workshop.select_one('var.atc_description').text

' Unfamiliar with patents, trademarks, copyrights, and trade secrets?\xa0Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and potential ways to protect your innovation as you transition from idea to product.\xa0\nThe USPTO offers this free event for aspiring entrepreneurs, innovators, and students the first Thursday of every month.\xa0\nRegister today\xa0\nThe discussion will cover:\nAn overview of intellectual property types: patents, trademarks, copyrights, and trade secrets\tWhy innovators and entrepreneurs should consider protecting their IP\tLocal resources and assistance available through the USPTO and other agencies\nFor questions or more information, email\xa0Events@uspto.gov. '

In [61]:
tr[index]

<tr class="type-public-events type-guidance---training topic-alexandria-va-headquarters topic-dallas topic-denver topic-silicon-valley topic-detroit topic-all-topics type-all-event-types year-now year-2022" date="20221201"><td class="day views-field views-field-field-date-range-2" headers="view-field-date-range-2-table-column--96"><div>1</div>
</td>
<td class="name views-field views-field-title"><a href="/about-us/events/intellectual-property-basics-and-helpful-resources-10" hreflang="en">Intellectual property basics and helpful resources</a> </td>
<td class="location views-field views-field-field-address">Virtual
          </td>
<td class="time views-field views-field-field-date-range-3" headers="view-field-date-range-3-table-column--96"><div>12:00 p.m. ET</div>
</td>
</tr>

In [114]:
import time

In [92]:
time.sleep(6)

In [53]:
import time
import pandas as pd

In [80]:
title = soup_workshop.find('meta', {"name":"dcterms.title"})
title["content"]

'Intellectual property basics and helpful resources'

In [82]:
workshop_description = soup_workshop.select_one('var.atc_description').text
workshop_description = workshop_description[0:workshop_description.find('\n')].strip().replace('\xa0', ' ')
print(workshop_description)

Unfamiliar with patents, trademarks, copyrights, and trade secrets? Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and potential ways to protect your innovation as you transition from idea to product.


In [83]:
workshop_time = soup_workshop.select_one('var.atc_date_start').text
workshop_time = workshop_time.replace(" ", "T")
workshop_time = datetime.datetime.fromisoformat(workshop_time)
print(workshop_time.strftime("%a, %d %b %Y %X EST"))

Thu, 20 Jan 2022 20:00:00 EST


In [26]:
        workshop_title = soup_workshop.find('meta', {"name":"dcterms.title"})
        workshop_pageowner = soup_workshop.find('meta', {"name":"uspto.pageowner"})
        
        workshop_description = soup_workshop.select_one('var.atc_description').text
        workshop_description = workshop_description[0:workshop_description.find('\n')].strip().replace('\xa0', ' ')
        workshop_description = workshop_description.replace('\u200b','')

        workshop_time = soup_workshop.select_one('var.atc_date_start').text
        workshop_time = workshop_time.replace(" ", "T")
        workshop_time = dt.fromisoformat(workshop_time)


In [28]:
        string = str(tr[index])
        date_index=(string.find("date="))+6
#        print(date_index)
#        print("iso8601 date: ",string[date_index:date_index+8])
        date = int(string[date_index:date_index+8])
        workshop_url = tr[index].find(href=True)['href']


NameError: name 'tr' is not defined

In [29]:
        workshop_info = pd.DataFrame({
            "title": [workshop_title["content"]], 
            "link": [url_start+workshop_url], 
            "pubDate": [workshop_time.strftime("%a, %d %b %Y %X EST")], 
            "description": [workshop_description],
            "pageowner": [workshop_pageowner["content"]], 
            "date_index": ['20220125']
        })        


In [30]:
workshop_info

,title,link,pubDate,description,pageowner,date_index
0,Intellectual property basics and helpful resou...,https://www.uspto.gov/about-us/events/intellec...,"Thu, 01 Dec 2022 12:00:00 EST","Unfamiliar with patents, trademarks, copyright...",All Regions,20220125


In [104]:
new = [events, workshop_info]
events = pd.concat(new)

In [94]:
workshop_time.strftime("%a, %d %b %Y %X EST")


'Thu, 20 Jan 2022 20:00:00 EST'

In [95]:
workshop_description

'Unfamiliar with patents, trademarks, copyrights, and trade secrets? Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and potential ways to protect your innovation as you transition from idea to product.'

In [99]:
%whos

Variable               Type             Data/Info
-------------------------------------------------
BeautifulSoup          type             <class 'bs4.BeautifulSoup'>
current_date           int              20221231
date                   int              20221201
date_index             int              213
datetime               module           <module 'datetime' from '<...>onda3\\lib\\datetime.py'>
events                 DataFrame        Empty DataFrame\nColumns:<...>, description]\nIndex: []
headers                dict             n=1
index                  int              3036
pd                     module           <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests               module           <module 'requests' from '<...>\\requests\\__init__.py'>
soup                   BeautifulSoup    <!DOCTYPE html>\n\n<html <...></script></body></html>\n
soup_workshop          BeautifulSoup    <!DOCTYPE html>\n<html di<...>"></script></body></html>
string                 st

In [130]:
datetime.datetime.now().strftime("%a, %d %b %Y %X PST")

'Mon, 24 Jan 2022 21:30:32 PST'

In [127]:
datetime.datetime.now()

datetime.datetime(2022, 1, 24, 21, 27, 8, 162292)

In [164]:
xml_header = "<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<rss version=\"2.0\">\n<channel>\n<title>USPTO Patent Events</title>\n<link>https://guides.lib.ucr.edu/c.php?g=932732</link>\n<description>This feed provides easy access to USPTO Patents events.</description>\n<language>en-us</language>\n<lastBuildDate>"+datetime.datetime.now().strftime("%a, %d %b %Y %X PST")+"</lastBuildDate>\n<webMaster>katherine.koziar@ucr.edu</webMaster>"

In [132]:
xml_header

'<?xml version="1.0" encoding="utf-8"?>\n<rss version="2.0">\n<channel>\n<title>USPTO Patent Events</title>\n<link>https://guides.lib.ucr.edu/c.php?g=932732</link>\n<description>This feed provides easy access to USPTO Patents events.</description>\n<language>en-us</language>\n<lastBuildDate>Mon, 24 Jan 2022 21:30:51 PST</lastBuildDate>\n<webMaster>katherine.koziar@ucr.edu</webMaster>'

# create xml file

In [103]:
xml_files = ["All", "Patent", "Trademark", "Other"]

In [120]:
xml_dict = {"All": ["<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<rss version=\"2.0\">\n<channel>\n"], 
             "Patent": ["<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<rss version=\"2.0\">\n<channel>\n"],
             "Trademark": ["<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<rss version=\"2.0\">\n<channel>\n"] , 
             "Other": ["<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<rss version=\"2.0\">\n<channel>\n"]
            }

In [121]:
for category in xml_files:
    xml_dict[category].append("<title>USPTO "+category+" Events</title>\n<link>https://guides.lib.ucr.edu/c.php?g=932732</link>\n")
    xml_dict[category].append("<description>This feed provides easy access to USPTO "+category+" events.</description>\n<language>en-us</language>\n")    
    xml_dict[category].append("<lastBuildDate>"+dt.now().strftime("%a, %d %b %Y %X PST")+"</lastBuildDate>\n<webMaster>katherine.koziar@ucr.edu</webMaster>\n\n")        

In [122]:
for i in range(len(events)):
    title = "\n<item> <title>"+events.iloc[i, 0]+"</title>\n"
    link = "\t<link>"+events.iloc[i,1]+"</link>\n"
    guid = "\t<guid>"+events.iloc[i,1]+"</guid>\n"
    pubDate = "\t<pubDate>"+events.iloc[i,2]+"</pubDate>\n"
    description = "\t<description>"+events.iloc[i,3]+"</description>\n"
    description = description.replace("’","'")
    enditem = "</item>\n\n"
    
    xml_dict["All"].append(title+link+guid+pubDate+description+enditem)
    
    if(events.iloc[i,4] == 'All Regions') or (events.iloc[i,4] == 'ecommerce'):
        xml_dict["Patent"].append(title+link+guid+pubDate+description+enditem)
    elif (events.iloc[i,4] == 'Trademarks'):
        xml_dict["Trademark"].append(title+link+guid+pubDate+description+enditem)
    else:
        xml_dict["Other"].append(title+link+guid+pubDate+description+enditem)


In [119]:
for category in xml_files:
    xml_dict[category].append("\n</channel>\n</rss>")    
    temp = " " 
    temp = temp.join(xml_dict[category])
    xml_file = open("../rss-"+category.lower()+".xml", "w")
    xml_file.write(temp)
    xml_file.close()

In [99]:
for category in xml_files:
    print(category.lower())

all
patent
trademark
other


In [14]:
xml_close = "\n</channel>\n</rss>"
xml_all = xml_all+xml_close

In [62]:
xml_dict[category].append()

In [15]:
xml_file = open("../test.xml", "w")
xml_file.write(xml)
xml_file.close()

In [88]:
xml_dict

{'All': ['',
  '<?xml version="1.0" encoding="utf-8"?>\n<rss version="2.0">\n<channel>\n',
  '<title>USPTO All Events</title>\n<link>https://guides.lib.ucr.edu/c.php?g=932732</link>\n',
  '<description>This feed provides easy access to USPTO All Patents events.</description>\n<language>en-us</language>\n',
  '<lastBuildDate>Tue, 25 Jan 2022 11:40:19 PST</lastBuildDate>\n<webMaster>katherine.koziar@ucr.edu</webMaster>\n\n',
  '<item> <title>Intellectual property basics and helpful resources</title>\n\t<link>https://www.uspto.gov/about-us/events/intellectual-property-basics-and-helpful-resources-11</link>\n\t<guid>https://www.uspto.gov/about-us/events/intellectual-property-basics-and-helpful-resources-11</guid>\n\t<pubDate>Thu, 20 Jan 2022 20:00:00 EST</pubDate>\n\t<description>Unfamiliar with patents, trademarks, copyrights, and trade secrets? Don’t miss this opportunity to learn from United States Patent and Trademark Office (USPTO) experts about intellectual property (IP) basics and p

In [174]:
events

,title,link,pubDate,description,date_index
0,Intellectual property basics and helpful resou...,https://www.uspto.gov/about-us/events/intellec...,"Thu, 20 Jan 2022 20:00:00 EST","Unfamiliar with patents, trademarks, copyright...",20220120
0,Intellectual property (IP) workshop for K-12 e...,https://www.uspto.gov/learning-and-resources/k...,"Tue, 25 Jan 2022 18:00:00 EST",,20220125
0,Attend DOCX filing training,https://www.uspto.gov/about-us/events/attend-d...,"Tue, 25 Jan 2022 13:00:00 EST",Learn how to file patent-related documents in ...,20220125
0,"Trademark Basics Boot Camp, Module 3: Searching",https://www.uspto.gov/about-us/events/trademar...,"Tue, 25 Jan 2022 14:00:00 EST",Are you a small business owner or entrepreneur...,20220125
0,USPTO's liquid capital: Data resources for inv...,https://www.uspto.gov/about-us/events/usptos-l...,"Wed, 26 Jan 2022 13:00:00 EST",The United States Patent and Trademark Office'...,20220126
...,...,...,...,...,...
0,Intellectual property basics and helpful resou...,https://www.uspto.gov/about-us/events/intellec...,"Thu, 06 Oct 2022 12:00:00 EST","Unfamiliar with patents, trademarks, copyright...",20221006
0,Intellectual Property (IP) Workshop for K12 Ed...,https://www.uspto.gov/learning-and-resources/k...,"Tue, 25 Oct 2022 18:00:00 EST",,20221025
0,K-12 IP education office hours,https://www.uspto.gov/learning-and-resources/k...,"Wed, 26 Oct 2022 15:00:00 EST",,20221026
0,Intellectual property basics and helpful resou...,https://www.uspto.gov/about-us/events/intellec...,"Thu, 03 Nov 2022 12:00:00 EST","Unfamiliar with patents, trademarks, copyright...",20221103


In [42]:
workshop_pageowner["content"]

'All Regions'

In [53]:
xml_files[0]

'All'

In [60]:
xml_dict.values

<function dict.values>

In [72]:
temp = " " 

In [75]:
temp = " " 
temp = temp.join(xml_dict[xml_files[0]])

In [76]:
temp

'one three'

temp